Cuardeno para procesar los programas snies
https://hecaa.mineducacion.gov.co/consultaspublicas/programas

### Constantes y Librerias 

In [1]:
import pandas as pd
from unidecode import unidecode
import unicodedata
from rapidfuzz import process, fuzz
import re

import constants

### Funciones predefinidas

In [2]:
def quitar_tildes(texto):
    if pd.isna(texto):
        return texto
    # Convertir a unicode normalizado sin acentos
    texto = unicodedata.normalize('NFKD', str(texto))
    texto = ''.join([c for c in texto if not unicodedata.combining(c)])
    return texto

In [3]:
def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    # Convertir a string
    texto = str(texto)
    # Eliminar comillas dobles
    texto = texto.replace('"', '')
    # Eliminar backslashes
    texto = texto.replace('\\', '')
    # Eliminar espacios extra
    texto = ' '.join(texto.split())
    # Eliminar tildes y acentos
    texto = quitar_tildes(texto) 
    # Capitalizar tipo título
    texto = texto.title()
    
    return texto

In [4]:
def limpiar_texto_dataframe(df):
    # Limpiar el texto de las columnas tipo object (strings)
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].apply(limpiar_texto)

    return df

In [5]:
def limpiar_nombres_columnas(df):
    df.columns = df.columns.str.lower()
    df.columns = [quitar_tildes(col) for col in df.columns]
    
    return df

In [6]:
def normalizar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = str(texto).lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')  # quitar tildes
    texto = re.sub(r'[\W_]+', ' ', texto)  # eliminar puntuación y guiones, dejar solo letras/números
    texto = re.sub(r'\s+', ' ', texto).strip()  # eliminar espacios múltiples
    return texto

In [7]:
def unir_por_similitud(programas, municipios, col_prog='municipio_oferta_programa', col_mpio='nombre_municipio', threshold=90):
    """
    Realiza un join difuso entre los DataFrames 'programas' y 'municipios',
    comparando textos similares entre dos columnas, usando RapidFuzz (WRatio).

    Parámetros:
        programas (DataFrame): DataFrame principal (ej. programas académicos).
        municipios (DataFrame): DataFrame de referencia (ej. nombres estándar de municipios).
        col_prog (str): Columna de 'programas' a comparar.
        col_mpio (str): Columna de 'municipios' a comparar.
        threshold (int): Similitud mínima (0–100) para considerar un match (default: 90).

    Retorna:
        DataFrame resultante con las filas unidas según similitud textual.
    """
    
    # Normalizar columnas
    programas['__match_key'] = programas[col_prog].apply(normalizar_texto)
    municipios['__match_key'] = municipios[col_mpio].apply(normalizar_texto)

    resultados = []

    for val in programas['__match_key'].dropna().unique():
        match = process.extractOne(val, municipios['__match_key'].dropna().unique(), scorer=fuzz.WRatio)
        
        if match and match[1] >= threshold:
            prog_filtrado = programas[programas['__match_key'] == val]
            mpio_filtrado = municipios[municipios['__match_key'] == match[0]]

            # Ajuste para asegurar coincidencia de clave
            prog_filtrado = prog_filtrado.assign(__match_key=match[0])

            combinado = prog_filtrado.merge(mpio_filtrado, on='__match_key')
            resultados.append(combinado)

    # Eliminar columna auxiliar
    return pd.concat(resultados, ignore_index=True).drop(columns='__match_key') if resultados else pd.DataFrame()

In [8]:
def filtrar_programas(df):
    """
    Filtra el DataFrame para conservar solo los programas:
    - Con estado 'Activo'
    - De nivel de formación 'Universitario'
    - De nivel académico 'Pregrado'
    
    Parámetros:
        df (pd.DataFrame): El DataFrame original.
        
    Retorna:
        pd.DataFrame: El DataFrame filtrado.
    """
    return df[
        (df['estado_programa'] == "Activo") &
        ((df['nivel_de_formacion'] == "Formacion Tecnica Profesional") | (df['nivel_de_formacion'] == "Tecnologico")) &
        (df['nivel_academico'] == "Pregrado")
    ]

In [9]:
def filtrar_departamentos(df, departamentos):
    """
    Filtra el DataFrame por una lista de departamentos.
    
    Parámetros:
        df (pd.DataFrame): DataFrame original.
        departamentos (list): Lista de departamentos a filtrar.
        
    Retorna:
        pd.DataFrame: DataFrame filtrado.
    """
    return df[df['departamento_oferta_programa'].isin(departamentos)]


### Lectura de los datos

In [10]:
programas = pd.read_excel("../data/SNIES_CINE_raw/programas_nivel_nacional.xlsx")
programas = limpiar_nombres_columnas(programas)
programas = programas[constants.columnas_base_snies]
programas = limpiar_texto_dataframe(programas)

In [11]:
programas.head()

,codigo_institucion,codigo_institucion_padre,nombre_institucion,estado_institucion,caracter_academico,codigo_snies_del_programa,nombre_del_programa,titulo_otorgado,estado_programa,cine_f_2013_ac_campo_amplio,...,nivel_academico,nivel_de_formacion,modalidad,numero_creditos,numero_periodos_de_duracion,periodicidad,departamento_oferta_programa,municipio_oferta_programa,costo_matricula_estud_nuevos,reconocimiento_del_ministerio
0,1101,1101,Universidad Nacional De Colombia,Activa,Universidad,19,Administracion De Empresas,Administrador(A) De Empresas,Activo,Administracion De Empresas Y Derecho,...,Pregrado,Universitario,Presencial,164.0,10.0,Semestral,"Bogota, D.C.","Bogota, D.C.",NaN,Acreditacion De Alta Calidad
1,1101,1101,Universidad Nacional De Colombia,Activa,Universidad,13,Antropologia,Antropologo(A),Activo,Arte Y Humanidades,...,Pregrado,Universitario,Presencial,122.0,9.0,Semestral,"Bogota, D.C.","Bogota, D.C.",NaN,Acreditacion De Alta Calidad
2,1101,1101,Universidad Nacional De Colombia,Activa,Universidad,30,Arquitectura,Arquitecto(A),Activo,"Ingenieria, Industria Y Construccion",...,Pregrado,Universitario,Presencial,179.0,10.0,Semestral,"Bogota, D.C.","Bogota, D.C.",NaN,Acreditacion De Alta Calidad
3,1101,1101,Universidad Nacional De Colombia,Activa,Universidad,2497,Artes Plasticas,Maestro En Artes Plasticas,Activo,Arte Y Humanidades,...,Pregrado,Universitario,Presencial,166.0,10.0,Semestral,"Bogota, D.C.","Bogota, D.C.",NaN,Acreditacion De Alta Calidad
4,1101,1101,Universidad Nacional De Colombia,Activa,Universidad,31,Biologia,Biologo(A),Activo,"Ciencias Naturales, Matematicas Y Estadistica",...,Pregrado,Universitario,Presencial,163.0,10.0,Semestral,"Bogota, D.C.","Bogota, D.C.",NaN,Acreditacion De Alta Calidad


In [12]:
programas.shape

(30034, 24)

### Filtrar por Bogota-Region

In [13]:
municipios_bogota_region = pd.read_csv("../data/Municipios_cleaned/municipios.csv")

In [14]:
lista_deptos = ["Bogota, D.C.", "Cundinamarca"]

programas = filtrar_departamentos(programas, lista_deptos)
programas = unir_por_similitud(programas.copy(), municipios_bogota_region)
print(f"Programas en Bogota Region {programas.shape[0]}")

Programas en Bogota Region 8673


### Filtrar por programas tecnologicos o tecnicos profesionales activos y de pregrado

In [15]:
programas = filtrar_programas(programas)
print(f"Programas Tecnologicos o Tecnicos Profesionales en Bogota Region {programas.shape[0]}")

Programas Tecnologicos o Tecnicos Profesionales en Bogota Region 849



### Guardar Dataframe

In [20]:
#Guardar el dataframe como un archivo csv en la carpeta SNIES_CINE_cleaned
programas.to_csv("../data/SNIES_CINE_cleaned/base_cine_tyt.csv", index=False)